# Amazon Bedrock 모델 평가를 사용한 RAG 애플리케이션 구축 및 평가
이 가이드에서는 Amazon Bedrock Knowledge Bases의 Retrieve API와 LangChain을 사용하여 Q&A 애플리케이션을 구축하고, Amazon Bedrock의 내장 모델 평가 기능을 사용해 응답의 품질을 평가하는 방법을 알아봅니다.

기존 RAGAS 프레임워크 대신 Bedrock의 create_evaluation_job API를 사용하여 정확성(Correctness), 충실성(Faithfulness), 관련성(Relevancy) 등 RAG 애플리케이션에 필수적인 지표들을 평가합니다.

https://docs.aws.amazon.com/bedrock/latest/userguide/evaluation-kb.html

## Bedrock 모델 평가 워크플로우

**데이터셋 준비:** 
평가에 사용할 질문(prompt), 정답(referenceResponse), 검색된 컨텍스트(contexts), 그리고 모델의 생성 답변(modelOutput)이 포함된 .jsonl 형식의 데이터셋을 준비합니다.

**S3 업로드:** 
준비된 평가 데이터셋을 Amazon S3 버킷에 업로드합니다.

**평가 작업 생성:** 
create_evaluation_job Boto3 API를 호출하여 모델 평가 작업을 시작합니다. 이 작업은 S3에 저장된 데이터셋을 기반으로 지정된 모델의 응답 품질을 자동으로 평가합니다.

**결과 분석:** 
평가 작업이 완료되면 지정된 S3 버킷에 결과 리포트가 생성됩니다. 이 리포트를 통해 각 평가지표에 대한 점수를 확인하고 RAG 애플리케이션의 성능을 분석합니다.

## 전제 조건
**Knowledge Base:** Q&A에 사용할 데이터가 포함된 Amazon Bedrock Knowledge Base가 준비되어 있어야 합니다. (kb_id 필요)

**S3 버킷:** 평가용 데이터셋과 결과 리포트를 저장할 S3 버킷이 필요합니다.

**IAM 역할:** Amazon Bedrock이 S3 버킷에 접근하고 평가 작업을 수행할 수 있는 권한을 가진 IAM 역할이 필요합니다. 공식 문서를 참고하여 서비스 역할을 생성하세요.

## 1단계: RAG 파이프라인 실행 및 평가 데이터 생성
먼저, 기존 노트북과 동일하게 LangChain과 Bedrock Knowledge Base를 사용하여 각 질문에 대한 답변과 컨텍스트를 생성합니다. 이 결과물은 Bedrock 모델 평가를 위한 데이터셋의 기반이 됩니다.

In [1]:
import boto3
import json
from botocore.client import Config
from langchain.chains import RetrievalQA
from langchain_aws.chat_models.bedrock import ChatBedrock
from langchain_aws.retrievers.bedrock import AmazonKnowledgeBasesRetriever
import sagemaker

# --- 1. 기본 클라이언트 및 모델 설정 ---
# 기존 노트북의 설정 코드와 동일합니다.
session = boto3.session.Session()
region = session.region_name
bedrock_config = Config(connect_timeout=120, read_timeout=120, retries={'max_attempts': 0})
bedrock_client = boto3.client('bedrock-runtime', region_name=region)
bedrock_agent_client = boto3.client("bedrock-agent-runtime", config=bedrock_config, region_name=region)

# 평가할 모델과 Knowledge Base 리트리버 설정
llm_for_text_generation = ChatBedrock(model_id="anthropic.claude-3-haiku-20240307-v1:0", client=bedrock_client)

%store -r kb_id
# kb_id = "YOUR_KNOWLEDGE_BASE_ID" # Knowledge Base ID

# default S3 bucket 확인
sm_session = sagemaker.Session()
default_bucket = sm_session.default_bucket()

retriever = AmazonKnowledgeBasesRetriever(
    knowledge_base_id=kb_id,
    retrieval_config={"vectorSearchConfiguration": {"numberOfResults": 5}},
)

qa_chain = RetrievalQA.from_chain_type(
    llm=llm_for_text_generation, 
    retriever=retriever, 
    return_source_documents=True
)

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [2]:
from botocore.exceptions import ClientError, NoCredentialsError

# 수정할 S3 버킷 이름을 입력하세요
BUCKET_NAME = default_bucket

print(f"작업 대상 버킷: {BUCKET_NAME}")

작업 대상 버킷: sagemaker-us-west-2-461433424192


In [7]:
import boto3
s3_client = boto3.client('s3')

def get_cors_configuration(bucket_name):
    """
    S3 버킷의 현재 CORS 설정을 조회합니다.
    
    Args:
        bucket_name (str): S3 버킷 이름
    
    Returns:
        dict: CORS 설정 또는 None
    """
    try:
        response = s3_client.get_bucket_cors(Bucket=bucket_name)
        return response['CORSRules']
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == 'NoSuchCORSConfiguration':
            print(f"버킷 '{bucket_name}'에 CORS 설정이 없습니다.")
            return None
        else:
            print(f"CORS 설정 조회 중 오류 발생: {e}")
            return None
    except Exception as e:
        print(f"예상치 못한 오류 발생: {e}")
        return None

# 현재 CORS 설정 조회
current_cors = get_cors_configuration(BUCKET_NAME)
if current_cors:
    print("현재 CORS 설정:")
    print(json.dumps(current_cors, indent=2, ensure_ascii=False))
else:
    print("현재 CORS 설정이 없습니다.")

버킷 'sagemaker-us-west-2-461433424192'에 CORS 설정이 없습니다.
현재 CORS 설정이 없습니다.


In [8]:
# 새로운 CORS 설정 정의
cors_configuration = {
    'CORSRules': [
        {
            "AllowedHeaders": [
            "*"
            ],
            "AllowedMethods": [
                "GET",
                "PUT",
                "POST",
                "DELETE"
            ],
            "AllowedOrigins": [
                "*"
            ],
            "ExposeHeaders": [
                "Access-Control-Allow-Origin"
            ]
        }
    ]
}

print("새로운 CORS 설정:")
print(json.dumps(cors_configuration, indent=2, ensure_ascii=False))

새로운 CORS 설정:
{
  "CORSRules": [
    {
      "AllowedHeaders": [
        "*"
      ],
      "AllowedMethods": [
        "GET",
        "PUT",
        "POST",
        "DELETE"
      ],
      "AllowedOrigins": [
        "*"
      ],
      "ExposeHeaders": [
        "Access-Control-Allow-Origin"
      ]
    }
  ]
}


In [9]:
def put_cors_configuration(bucket_name, cors_config):
    """
    S3 버킷에 CORS 설정을 적용합니다.
    
    Args:
        bucket_name (str): S3 버킷 이름
        cors_config (dict): CORS 설정
    
    Returns:
        bool: 성공 여부
    """
    try:
        s3_client.put_bucket_cors(
            Bucket=bucket_name,
            CORSConfiguration=cors_config
        )
        print(f"버킷 '{bucket_name}'의 CORS 설정이 성공적으로 업데이트되었습니다.")
        return True
    except ClientError as e:
        print(f"CORS 설정 적용 중 오류 발생: {e}")
        return False
    except Exception as e:
        print(f"예상치 못한 오류 발생: {e}")
        return False

# CORS 설정 적용
success = put_cors_configuration(BUCKET_NAME, cors_configuration)
if success:
    print("\n✅ CORS 설정이 성공적으로 적용되었습니다!")
else:
    print("\n❌ CORS 설정 적용에 실패했습니다.")

버킷 'sagemaker-us-west-2-461433424192'의 CORS 설정이 성공적으로 업데이트되었습니다.

✅ CORS 설정이 성공적으로 적용되었습니다!


In [10]:
# 업데이트된 CORS 설정 확인
print("업데이트된 CORS 설정 확인:")
updated_cors = get_cors_configuration(BUCKET_NAME)
if updated_cors:
    print(json.dumps(updated_cors, indent=2, ensure_ascii=False))
else:
    print("CORS 설정을 찾을 수 없습니다.")

업데이트된 CORS 설정 확인:
[
  {
    "AllowedHeaders": [
      "*"
    ],
    "AllowedMethods": [
      "GET",
      "PUT",
      "POST",
      "DELETE"
    ],
    "AllowedOrigins": [
      "*"
    ],
    "ExposeHeaders": [
      "Access-Control-Allow-Origin"
    ]
  }
]


**평가 데이터셋 양식**

{"conversationTurns":[{"prompt":{"content":[{"text":"Provide the prompt you want to use during inference"}]},"referenceResponses":[{"content":[{"text":"Specify a ground-truth response"}]}]}]}

{"conversationTurns":[{"prompt":{"content":[{"text":"Provide the prompt you want to use during inference"}]},"referenceResponses":[{"content":[{"text":"Specify a ground-truth response"}]}]}]}

{"conversationTurns":[{"prompt":{"content":[{"text":"Provide the prompt you want to use during inference"}]},"referenceResponses":[{"content":[{"text":"Specify a ground-truth response"}]}]}]}

{"conversationTurns":[{"prompt":{"content":[{"text":"Provide the prompt you want to use during inference"}]},"referenceResponses":[{"content":[{"text":"Specify a ground-truth response"}]}]}]}

{"conversationTurns":[{"prompt":{"content":[{"text":"Provide the prompt you want to use during inference"}]},"referenceResponses":[{"content":[{"text":"Specify a ground-truth response"}]}]}]}

{"conversationTurns":[{"prompt":{"content":[{"text":"Provide the prompt you want to use during inference"}]},"referenceResponses":[{"content":[{"text":"Specify a ground-truth response"}]}]}]}

https://docs.aws.amazon.com/bedrock/latest/userguide/knowledge-base-evaluation-prompt.html

In [11]:
import json

# --- 1. 평가용 질문 및 정답 데이터 ---
questions = [
        "2024년 전국의 총인구와 전년 대비 증감률을 제시하고, 내국인과 외국인의 구성을 설명하시오.",
        "2024년 전국의 연령대별 인구 구성과 노령화지수를 제시하시오.",
        "2024년 다문화가구의 규모와 유형별 구성비를 설명하시오.",
        "2024년 (반)지하 및 옥탑 주택의 지역별 분포 특성을 설명하시오.",
        "2024년 전국 주택의 노후기간별 현황을 제시하시오."
]
ground_truths = [
    "2024년 총인구는 51,806천 명이며 전년 대비 0.1%(31천 명) 증가했습니다. 내국인: 49,763천 명(96.1%), 전년 대비 77천 명(-0.2%) 감소. 외국인: 2,043천 명(3.9%), 전년 대비 108천 명(5.6%) 증가",
    "2024년 연령대별 인구 구성은 다음과 같습니다: 유소년인구(0-14세): 542만 명(10.5%), 생산연령인구(15-64세): 3,626만 명(70.0%), 고령인구(65세 이상): 1,012만 명(19.5%), 노령화지수: 186.7(전년 대비 15.7 증가)",
    "2024년 다문화가구는 439,304가구이며, 유형별 구성은 다음과 같습니다: 귀화자가구: 42.6%, 결혼이민자가구: 35.2%, 다문화자녀가구: 11.3%, 기타: 10.9% 전년 대비 5.7%(23,720가구) 증가했습니다.",
    "2024년 기준: (반)지하가 있는 주택: 261천 호(전체 주택의 1.3%), 옥탑이 있는 주택: 34천 호(전체 주택의 0.2%), 수도권 분포 비율: (반)지하: 97.3%(254천 호) 옥탑: 90.6%(31천 호)",
    "2024년 주택의 노후기간별 현황: 20년 이상 된 주택: 10,908천 호(전체 주택의 54.9%) 20~30년 미만: 5,334천 호(26.8%) 30년 이상: 5,574천 호(28.0%)"
]

In [12]:
# --- 2. JSON Lines(.jsonl) 파일로 저장 ---
evaluation_data = []
for question, ground_truth in zip(questions, ground_truths):
    data = {
        "conversationTurns": [{
            "prompt": {
                "content": [{
                    "text": question
                }]
            },
            "referenceResponses": [{
                "content": [{
                    "text": ground_truth
                }]
            }]
        }]
    }
    evaluation_data.append(data)

output_file = "evaluation_dataset.jsonl"
with open(output_file, "w") as f:
    for item in evaluation_data:
        f.write(json.dumps(item) + "\n")

print(f"'{output_file}'이 성공적으로 생성되었습니다.")

'evaluation_dataset.jsonl'이 성공적으로 생성되었습니다.


## 2단계: S3 업로드
생성된 .jsonl 파일을 S3에 업로드한 후, create_evaluation_job API를 호출하여 평가를 시작합니다.

평가 모델로 사용가능한 llm 리스트 https://docs.aws.amazon.com/bedrock/latest/userguide/evaluation-judge.html

In [13]:
# --- 3. 데이터셋 S3 업로드 ---
s3_client = boto3.client('s3')
bucket_name = default_bucket  # S3 버킷 이름으로 변경하세요.
s3_key = "evaluation-datasets/evaluation_dataset.jsonl"
s3_input_path = f"s3://{bucket_name}/{s3_key}"

s3_client.upload_file(output_file, bucket_name, s3_key)
print(f"데이터셋이 {s3_input_path}에 업로드되었습니다.")

데이터셋이 s3://sagemaker-us-west-2-461433424192/evaluation-datasets/evaluation_dataset.jsonl에 업로드되었습니다.


## 3단계: Bedrock 모델 평가 작업 실행

### Amazon Bedrock RAG 평가 작업 생성 가이드

### 1. Amazon Bedrock 콘솔 접속
Amazon Bedrock 콘솔을 엽니다.

### 2. 평가 메뉴 접근
왼쪽 패널의 **Assessment** 섹션에서 **Evaluations**를 선택합니다.

### 3. RAG 평가 생성 시작
**RAG 평가** 패널에서 **생성(Create)**을 선택합니다.

### 4. RAG 평가 세부 정보 입력

#### 4.1 기본 정보 설정
- **평가 세부 정보** 패널의 **평가 이름** 필드에 평가 작업의 이름을 입력합니다.
  - 입력하는 이름은 AWS 리전 내에서 고유해야 합니다.
- **설명 - 선택사항** 필드에 평가 작업에 대한 설명을 입력합니다. (선택사항)

#### 4.2 평가자 모델 선택
**평가자 모델** 섹션에서 **모델 선택**을 클릭하고 평가를 수행할 평가자 모델을 선택합니다.

### 5. 추론 소스 설정

Amazon Bedrock RAG 평가에서는 두 가지 방법 중 하나를 선택할 수 있습니다:

#### 5.1 Amazon Bedrock Knowledge Base 사용
Amazon Bedrock Knowledge Base의 성능을 평가하려면:
- **추론 소스** 패널에서 **소스 선택**을 **Bedrock Knowledge Base**로 설정
- **Knowledge Base 선택** 드롭다운 목록에서 Knowledge Base를 선택

#### 5.2 자체 추론 응답 데이터 사용
프롬프트 데이터셋에서 자체 추론 응답 데이터를 제공하려면:
- **추론 소스** 패널에서 **소스 선택**을 **자체 추론 응답 가져오기(Bring your own inference responses)**로 설정
- **소스 이름**에 응답 데이터를 생성하는 데 사용한 RAG 소스의 이름을 입력
  - 입력하는 이름은 프롬프트 데이터셋의 `knowledgeBaseIdentifier` 매개변수와 일치해야 합니다

### 6. 평가 유형 선택
**추론 소스** 패널의 **평가 유형**에서 **검색 및 응답 생성(Retrieval and response generation)**을 선택합니다.

### 7. 메트릭 선택
**메트릭** 패널에서 평가자 모델이 사용할 내장 메트릭을 하나 이상 선택합니다.

### 8. 입력 및 출력 위치 정의

#### 8.1 데이터셋 설정
**데이터셋** 패널에서:
- **프롬프트 데이터셋 선택** 필드에 프롬프트 데이터셋의 Amazon S3 URI를 입력하거나
- **S3 찾아보기**를 선택하여 파일을 선택합니다
- 검색 전용 평가 작업에 필요한 프롬프트 데이터셋 형식에 대한 정의는 관련 문서를 참조하세요

#### 8.2 평가 결과 저장 위치
**평가 결과** 섹션에서:
- Amazon Bedrock이 결과를 저장할 Amazon S3 위치를 입력하거나
- **S3 찾아보기**를 선택하여 위치를 선택합니다

### 9. IAM 역할 설정
**Amazon Bedrock IAM 역할 - 권한** 섹션에서:
- **새 서비스 역할 생성 및 사용**: Amazon Bedrock이 평가 작업을 위한 새 IAM 역할을 생성하도록 하려면 선택
- **기존 서비스 역할 사용**: 기존 IAM 역할을 선택하려면 선택

평가 작업을 생성하고 실행하는 데 필요한 권한 목록은 전제 조건 문서를 참조하세요.

### 10. 암호화 설정 (선택사항)
자체 KMS 키를 사용하여 평가 데이터를 암호화하려면:
- **KMS 키 - 선택사항** 섹션에서 **암호화 설정 사용자 지정(고급)** 체크박스를 선택
- AWS KMS 키를 선택합니다

기본적으로 Amazon Bedrock은 AWS 소유 KMS 키로 평가 작업 데이터를 암호화합니다.

### 11. 평가 작업 생성 완료
**생성(Create)**을 선택하여 평가 작업 생성을 완료합니다.